In [1]:
# 혼공머신

In [2]:
# Chapter 05 트리 알고리즘

In [3]:
# 05-3 트리의 앙상블

In [4]:
# 앙상블 학습 : 정형 데이터(structured data)를 다루는 데 가장 뛰어난 성과를 내는 알고리즘
# 랜덤 포레스트 : 앙상블 학습의 대표주자
# 랜덤 포레스트는 결정 트리를 랜덤하게 만들어 결정 트리의 숲을 만들고, 
# 각 결정 트리의 에측을 사용해 최종 예측을 만듦
# 랜덤 포레스트는 입력된 훈련 데이터에서 랜덤하게 샘플 추출(중복 추출 가능. 부트스트랩 샘플)
# 노드를 분할할 때 전체 특성 중에서 일부 특성을 무작위로 고른 다음 이 중에서 최선의 분할을 찾음
# RandomForestClassifier는 전체 특성 개수의 제곱근만큼의 특성을 선택
# 랜덤 포레스트는 분류일 때는 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측으로 삼음
# 회귀일 떄는 단순히 각 트리의 예측을 평균
# 랜덤하게 선택한 샘플과 특성을 사용하기에 훈련세트에 과대적합되는 것을 막아주고 검증 세트와 테스트 세트에서 안정적 성능 얻을 수 있음

In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import  train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [9]:
#return_trian_score : 훈련 세트에 대한 점수도 같이 반환
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [10]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [11]:
# RandomForestClassifier에서는 자체적으로 모델을 평가하는 점수를 얻을 수 있음
# 부트스트랩 샘플에 포함되지 않고 남는 OOB 샘플을 사용하여 부트스트랩 샘플로 훈련한 결과를 평가할 수 있음
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


In [13]:
# 엑스트라 트리 : 전체 훈련 세트 사용. 무작위로 분할
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [15]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


In [16]:
# 그레이디언트 부스팅 : 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 함
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [17]:
# n_estimators : 결정 트리 개수
# learning_rate : 학습률
gb = GradientBoostingClassifier(n_estimators=500, learning_rate =0.2,
                                random_state=42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [18]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


In [20]:
# 히스토그램 기반 그레이디언트 부스팅
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target,
                        return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [22]:
# permutation_importance() : 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지를 관찰하여 어떤 특성이 중요한지 계산
# n_repeats : 랜덤하게 섞을 횟수 지정
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target,
                                n_repeats =10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [23]:
result = permutation_importance(hgb, test_input, test_target,
                                n_repeats =10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [24]:
hgb.score(test_input, test_target)

0.8723076923076923

In [26]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target,
                        return_train_score=True)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284


In [28]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(tree_method='hist', random_state=42)
scores = cross_validate(lgb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293
